## Preparation of final dataset
1. Loading files for Transport and Industry
2. Process industry data per subsector (Steel, Basic Chemicals, NMM)
3. Process transport data per subsectors and fuels (Electricity, Hydrogen, Gases, Liquids)
4. Prepare final excel file with one per each year and country merging all values

In [1178]:
import pandas as pd
from collections import defaultdict
import os

1. Loading files for Transport and Industry

In [1179]:
transport_file = os.path.join('..', 'REMIND', 'Results_REMIND_JRC.csv')
industry_path = os.path.join('Industry', 'Results_per_Country')

In [1180]:
country_code = "EU27"
# ['EU27', 'AT', 'BE', 'BG', 'CY', 
# 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI', 'FR', 'HR', 
# 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 
# 'PT', 'RO', 'SE', 'SI', 'SK']

years = [2030, 2040, 2050]

In [1181]:
output_path = os.path.join('..', 'Outputs', f'PtX_demand_{country_code}.xlsx')

In [1182]:
def load_transport_data(filepath):
    df = pd.read_csv(filepath)
    df['Year'] = df['Year'].astype(int)
    return df

In [1183]:
def load_industry_data(filepath):
    industry_data = []
    industry_files = [f for f in os.listdir(filepath) if f.endswith(".xlsx")]

    for file_name in industry_files:
        year, country = file_name.replace(".xlsx", "").split("_")
        file_path = os.path.join(filepath, file_name)
        df = pd.read_excel(file_path, index_col=0)
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)

        for material in df.index:
            for sector in df.columns:
                industry_data.append({
                    "Year": int(year),
                    "Country": country,
                    "Category": sector,
                    "Material": material.strip(),
                    "Value": df.loc[material, sector] * 3.6 * 0.000001
                })

    return pd.DataFrame(industry_data)

In [1184]:
transport_df = load_transport_data(transport_file)
industry_df = load_industry_data(industry_path)

2. Process industry data per subsector (Steel, Basic Chemicals, NMM)

In [1185]:
industry_mapping = {
    'Overall Demand': 'Overall Demand',
    'Hydrogen': 'Hydrogen',
    'Ammonia': 'Ammonia',
    'Methanol': 'Methanol',
    'Biomass': 'Biomass [Solid]',
    'Biogas': 'Biogenic Gases',
    'Other': 'Renewable Energy Carrier',
}

In [1186]:
industry_output = defaultdict(lambda: defaultdict(float))

In [1187]:
industry_filtered = industry_df[industry_df["Country"] == country_code]

In [1188]:

for _, row in industry_filtered.iterrows():
    year = row["Year"]
    mat = row["Material"]
    fuel = industry_mapping.get(mat)
    if fuel:
        key = (fuel, year)
        industry_output[key][row["Category"]] += row["Value"]

In [1189]:
industry_rows = []

for (fuel_group, year), sectors in industry_output.items():
    for sector, value in sectors.items():
        industry_rows.append({
            "FuelGroup": fuel_group,
            "Year": year,
            "IndustrySector": sector,
            "Value": value if value != 0 else None 
        })

industry_df = pd.DataFrame(industry_rows)

In [1190]:
pivot_industry = industry_df.pivot_table(
    index=["FuelGroup", "Year"],
    columns="IndustrySector",
    values="Value",
    aggfunc="sum"
).reset_index()
pivot_industry

IndustrySector,FuelGroup,Year,Chemicals,Iron & steel,Non-metallic minerals
0,Ammonia,2030,0.260617,0.000000,0.000000
1,Ammonia,2040,0.280098,0.000000,0.000000
2,Ammonia,2050,0.299502,0.000000,0.000000
3,Biogenic Gases,2030,0.000000,0.000000,0.031425
4,Biogenic Gases,2040,0.000000,0.000000,0.104480
5,Biogenic Gases,2050,0.000000,0.000000,0.211071
6,Biomass [Solid],2030,0.000000,0.000000,0.191637
7,Biomass [Solid],2040,0.000000,0.000000,0.180136
8,Biomass [Solid],2050,0.000000,0.000000,0.156159
9,Hydrogen,2030,0.000000,0.047676,0.000000


3. Process transport data per subsectors and fuels (Electricity, Hydrogen, Gases, Liquids)

In [1191]:
fuel_map = {
    'Liquids|Biomass': 'Biogenic Liquids',
    'Gases|Biomass': 'Biogenic Gases',
    'Liquids|Hydrogen': 'Synthetic Liquids',
    'Gases|Hydrogen': 'Synthetic Gases',
    'Hydrogen': 'Hydrogen',
}


In [1192]:
transport_output = defaultdict(lambda: defaultdict(float))

In [1193]:
categories = [
    "FE|Transport|Freight|Road|Heavy",
    "FE|Transport|Freight|Road|Light",
    "FE|Transport|Pass|Road|Bus",
    "FE|Transport|Pass|Road|LDV|Four Wheelers",
    "FE|Transport|Pass|Road|LDV|Two Wheelers",
    "FE|Transport|Pass|Domestic Aviation",
    "FE|Transport|Pass|Aviation",
    "FE|Transport|Pass|Rail",
    "FE|Transport|Freight|Rail",
    "FE|Transport|Bunkers|Freight|International Shipping",
    "FE|Transport|Freight|Domestic Shipping"               
]

transport_fuel_paths = [
    # Freight Road Heavy
    "FE|Transport|Freight|Road|Heavy|Electricity",
    "FE|Transport|Freight|Road|Heavy|Hydrogen",
    "FE|Transport|Freight|Road|Heavy|Gases",
    "FE|Transport|Freight|Road|Heavy|Gases|Biomass",
    "FE|Transport|Freight|Road|Heavy|Gases|Fossil",
    "FE|Transport|Freight|Road|Heavy|Gases|Hydrogen",
    "FE|Transport|Freight|Road|Heavy|Liquids",
    "FE|Transport|Freight|Road|Heavy|Liquids|Biomass",
    "FE|Transport|Freight|Road|Heavy|Liquids|Fossil",
    "FE|Transport|Freight|Road|Heavy|Liquids|Hydrogen",

    # Freight Road Light
    "FE|Transport|Freight|Road|Light|Electricity",
    "FE|Transport|Freight|Road|Light|Hydrogen",
    "FE|Transport|Freight|Road|Light|Gases",
    "FE|Transport|Freight|Road|Light|Gases|Biomass",
    "FE|Transport|Freight|Road|Light|Gases|Fossil",
    "FE|Transport|Freight|Road|Light|Gases|Hydrogen",
    "FE|Transport|Freight|Road|Light|Liquids",
    "FE|Transport|Freight|Road|Light|Liquids|Biomass",
    "FE|Transport|Freight|Road|Light|Liquids|Fossil",
    "FE|Transport|Freight|Road|Light|Liquids|Hydrogen",

    # Passenger Road Bus
    "FE|Transport|Pass|Road|Bus|Electricity",
    "FE|Transport|Pass|Road|Bus|Hydrogen",
    "FE|Transport|Pass|Road|Bus|Gases",
    "FE|Transport|Pass|Road|Bus|Gases|Biomass",
    "FE|Transport|Pass|Road|Bus|Gases|Fossil",
    "FE|Transport|Pass|Road|Bus|Gases|Hydrogen",
    "FE|Transport|Pass|Road|Bus|Liquids",
    "FE|Transport|Pass|Road|Bus|Liquids|Biomass",
    "FE|Transport|Pass|Road|Bus|Liquids|Fossil",
    "FE|Transport|Pass|Road|Bus|Liquids|Hydrogen",

    # Passenger Road LDV Four Wheelers
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Electricity",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Hydrogen",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Gases",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Gases|Biomass",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Gases|Fossil",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Gases|Hydrogen",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids|Biomass",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids|Fossil",
    "FE|Transport|Pass|Road|LDV|Four Wheelers|Liquids|Hydrogen",

    # Passenger Road LDV Two Wheelers
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Electricity",
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Liquids",
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Liquids|Biomass",
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Liquids|Fossil",
    "FE|Transport|Pass|Road|LDV|Two Wheelers|Liquids|Hydrogen",

    # Bunkers Freight International Shipping
    "FE|Transport|Bunkers|Freight|International Shipping|Liquids",

    # Freight Domestic Shipping
    "FE|Transport|Freight|Domestic Shipping|Liquids",
    "FE|Transport|Freight|Domestic Shipping|Liquids|Biomass",
    "FE|Transport|Freight|Domestic Shipping|Liquids|Fossil",
    "FE|Transport|Freight|Domestic Shipping|Liquids|Hydrogen",

    # Bunkers Pass International Aviation
    "FE|Transport|Bunkers|Pass|International Aviation|Liquids",

    # Passenger Domestic Aviation
    "FE|Transport|Pass|Domestic Aviation|Hydrogen",
    "FE|Transport|Pass|Domestic Aviation|Liquids",
    "FE|Transport|Pass|Domestic Aviation|Liquids|Biomass",
    "FE|Transport|Pass|Domestic Aviation|Liquids|Fossil",
    "FE|Transport|Pass|Domestic Aviation|Liquids|Hydrogen",

    # Passenger Aviation
    "FE|Transport|Pass|Aviation|Hydrogen",
    "FE|Transport|Pass|Aviation|Liquids",
    "FE|Transport|Pass|Aviation|Liquids|Biomass",
    "FE|Transport|Pass|Aviation|Liquids|Fossil",
    "FE|Transport|Pass|Aviation|Liquids|Hydrogen",

    # Passenger Rail
    "FE|Transport|Pass|Rail|Hydrogen",
    "FE|Transport|Pass|Rail|Liquids",
    "FE|Transport|Pass|Rail|Liquids|Biomass",
    "FE|Transport|Pass|Rail|Liquids|Fossil",
    "FE|Transport|Pass|Rail|Liquids|Hydrogen",

    # Freight Rail
    "FE|Transport|Freight|Rail|Hydrogen",
    "FE|Transport|Freight|Rail|Liquids", 
    "FE|Transport|Freight|Rail|Liquids|Biomass",
    "FE|Transport|Freight|Rail|Liquids|Fossil",
    "FE|Transport|Freight|Rail|Liquids|Hydrogen",
]

In [1194]:
def extract_main_and_fuel(category_str, categories):
    categories_sorted = sorted(categories, key=len, reverse=True)
    
    for cat_prefix in categories_sorted:
        if category_str.startswith(cat_prefix):
            # Fuel is whatever comes after the prefix (skip the '|')
            remainder = category_str[len(cat_prefix):]
            if remainder.startswith("|"):
                remainder = remainder[1:]  # remove leading '|'
            return cat_prefix, remainder
    # If no prefix matched, return None and full string as fuel
    return None, category_str

In [1195]:
transport_filtered = transport_df[(transport_df["Country"] == country_code) & 
                                  (transport_df["Year"].isin(years))]

In [1196]:
transport_filtered[["MainCategory", "Fuel"]] = transport_filtered["Category"].apply(lambda x: pd.Series(extract_main_and_fuel(x, categories)))
fuel_transport = transport_filtered[(transport_filtered["Fuel"].notna()) & 
                              (transport_filtered["Country"] == country_code)]

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_19848\1015843666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transport_filtered[["MainCategory", "Fuel"]] = transport_filtered["Category"].apply(lambda x: pd.Series(extract_main_and_fuel(x, categories)))
C:\Users\mar.eco\AppData\Local\Temp\ipykernel_19848\1015843666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transport_filtered[["MainCategory", "Fuel"]] = transport_filtered["Category"].apply(lambda x: pd.Series(extract_main_and_fuel(x, categories

In [1197]:
main_category_mapping = {
    "FE|Transport|Freight|Road|Heavy": "Freight Road",
    "FE|Transport|Freight|Road|Light": "Freight Road",
    "FE|Transport|Pass|Road|Bus": "Pass Road",
    "FE|Transport|Pass|Road|LDV|Four Wheelers": "Pass Road",
    "FE|Transport|Pass|Road|LDV|Two Wheelers": "Pass Road",
    "FE|Transport|Pass|Domestic Aviation": "Pass Aviation",
    "FE|Transport|Pass|Aviation": "Pass Aviation",
    "FE|Transport|Pass|Rail": "Pass Rail",
    "FE|Transport|Freight|Rail": "Freight Rail",
    "FE|Transport|Bunkers|Freight|International Shipping": "Maritime",
    "FE|Transport|Freight|Domestic Shipping": "Maritime",
}

In [1198]:
fuel_transport["TransportSector"] = fuel_transport["MainCategory"].map(main_category_mapping)

In [1199]:
fuel_transport["FuelGroup"] = fuel_transport["Fuel"].map(fuel_map)

In [1200]:
fuel_transport = fuel_transport[fuel_transport["FuelGroup"].notna()]

In [1201]:
agg_transport = fuel_transport.groupby(
    ["Year", "FuelGroup", "TransportSector"]
)["Value"].sum().reset_index()

In [1202]:
pivot_transport = agg_transport.pivot_table(
    index=["FuelGroup", "Year"],
    columns="TransportSector",
    values="Value",
    aggfunc="sum"
).reset_index()
pivot_transport

TransportSector,FuelGroup,Year,Freight Rail,Freight Road,Maritime,Pass Aviation,Pass Rail,Pass Road
0,Biogenic Gases,2030,NaN,2.767779e-03,NaN,NaN,NaN,6.850337e-03
1,Biogenic Gases,2040,NaN,4.540563e-03,NaN,NaN,NaN,8.690390e-03
2,Biogenic Gases,2050,NaN,1.288099e-03,NaN,NaN,NaN,1.368814e-03
3,Biogenic Liquids,2030,7.761598e-03,5.056034e-01,1.942107e-01,1.965263e-02,5.233833e-03,4.601055e-01
4,Biogenic Liquids,2040,9.429082e-03,3.367744e-01,2.230999e-01,3.051330e-02,8.532011e-03,1.976867e-01
5,Biogenic Liquids,2050,1.212729e-02,8.705103e-02,3.162198e-01,4.820971e-02,1.524521e-02,2.555552e-02
6,Hydrogen,2030,NaN,1.393642e-02,NaN,9.650719e-08,NaN,2.564214e-02
7,Hydrogen,2040,NaN,2.067317e-02,NaN,8.078718e-06,NaN,1.238285e-01
8,Hydrogen,2050,NaN,3.305030e-02,NaN,1.369278e-05,NaN,1.714660e-01
9,Synthetic Gases,2030,NaN,0.000000e+00,NaN,NaN,NaN,0.000000e+00


4. Prepare final excel file with one per each year and country merging all values

In [1203]:
pivot_transport = pivot_transport.rename(columns={"TransportSector": "SectorGroup"})
pivot_industry = pivot_industry.rename(columns={"FuelGroup": "FuelGroup", "Year": "Year"})

In [1204]:
pivot_industry["SectorGroup"] = "Industry"
pivot_transport["SectorGroup"] = "Transport"

In [1205]:
column_order = [
    "Iron & steel", "Chemicals", "Non-metallic minerals",  # Industry
    "Pass Road", "Pass Rail", "Pass Aviation",             # Transport (passenger)
    "Freight Road", "Freight Rail",                        # Transport (freight)
    "Maritime"                                             # Maritime (bunkers)
]

In [1206]:
all_columns = set(column_order + ["FuelGroup", "Year", "SectorGroup"])

# Add missing columns as None
for col in all_columns:
    if col not in pivot_transport.columns:
        pivot_transport[col] = None
    if col not in pivot_industry.columns:
        pivot_industry[col] = None

In [1207]:
combined_df = pd.concat([pivot_transport, pivot_industry], ignore_index=True)

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_19848\4054876749.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([pivot_transport, pivot_industry], ignore_index=True)


In [1208]:
# Ignore Overall Demand in this step
fuel_order = [
    "Hydrogen", "Methanol", "Ammonia", "Synthetic Gases",
    "Biogenic Gases", "Synthetic Liquids", "Biogenic Liquids",
    "Biomass [Solid]", "Renewable Energy Carrier"
]

In [1209]:
for year in [2030, 2040, 2050]:
    # Calculate the sum for EVERY column (Industry AND Transport) for that year
    sector_totals = combined_df[(combined_df["Year"] == year) & 
                             (combined_df["FuelGroup"].isin(fuel_order))][column_order].sum()
    
    # Place that sum into the "Overall Demand" row for that year
    combined_df.loc[(combined_df["Year"] == year) & 
                 (combined_df["FuelGroup"] == "Overall Demand"), column_order] = sector_totals.values

In [1210]:
fuel_order_map = {fuel: i for i, fuel in enumerate(fuel_order)}
combined_df["FuelSortOrder"] = combined_df["FuelGroup"].map(fuel_order_map)

In [1211]:
final_df = combined_df.drop(columns=["SectorGroup", "FuelSortOrder"])  # drop SectorGroup
final_df = final_df.groupby(["FuelGroup", "Year"], as_index=False).sum()

In [1212]:
def clean_zeros_and_nans(df):
    return df.applymap(lambda x: "" if pd.isna(x) or x == 0 else x)

In [1213]:
clean_df = clean_zeros_and_nans(final_df)

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_19848\2748423523.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: "" if pd.isna(x) or x == 0 else x)


In [1214]:
ordered_cols = ["FuelGroup", "Year"] + column_order
final_correct_order = clean_df[ordered_cols]
final_correct_order["FuelSortOrder"] = final_correct_order["FuelGroup"].map(fuel_order_map)
final_correct_order = final_correct_order.sort_values(by=["Year", "FuelSortOrder"]).drop(columns=["FuelSortOrder"])
final_correct_order

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_19848\2721408278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_correct_order["FuelSortOrder"] = final_correct_order["FuelGroup"].map(fuel_order_map)


,FuelGroup,Year,Iron & steel,Chemicals,Non-metallic minerals,Pass Road,Pass Rail,Pass Aviation,Freight Road,Freight Rail,Maritime
12,Hydrogen,2030,0.047676,,,0.025642,,0.0,0.013936,,
15,Methanol,2030,,0.063847,,,,,,,
0,Ammonia,2030,,0.260617,,,,,,,
24,Synthetic Gases,2030,,,,,,,,,
3,Biogenic Gases,2030,,,0.031425,0.00685,,,0.002768,,
27,Synthetic Liquids,2030,,,,,,,,,
6,Biogenic Liquids,2030,,,,0.460105,0.005234,0.019653,0.505603,0.007762,0.194211
9,Biomass [Solid],2030,,,0.191637,,,,,,
21,Renewable Energy Carrier,2030,,,0.025331,,,,,,
18,Overall Demand,2030,0.047676,0.324463,0.248392,0.492598,0.005234,0.019653,0.522308,0.007762,0.194211


In [1215]:
final_correct_order.to_excel(output_path, index=False, na_rep="")